# Working with Queries

The `Query` operation finds items based on primary key values. Partition key must be provided as equal condition and optionally sort key can be provided in `KeyConditionExpression` parameter to refine the results. Additionally you use `FilterExpression` and `Limit` to narrow down the results.

You can use `Query` from table and indexes as well. To use an index for querying, `IndexName` should be provided. If not, DynamoDB always fetch data from the base table.

In [7]:
# import and get dynamodb resource
import boto3
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
from pprint import pprint
from decimal import Decimal

dynamodb = boto3.resource('dynamodb')
starbucks = dynamodb.Table('Starbucks')

In [ ]:
items = []

with open('data/directory.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(
        f,
        fieldnames=['Brand', 'StoreNumber', 'StoreName', 'OwnershipType', 'StreetAddress', 'City', 'State', 'Country', 'Postcode', 'PhoneNumber', 'Timezone', 'Longitude', 'Latitude']
    )
    next(reader, None)
    
    for row in reader:
        item = {key: value for key, value in row.items() if value != ''}
        try:
            item['Longitude'] = Decimal(item['Longitude'])
            item['Latitude'] = Decimal(item['Latitude'])
        except:
            pass
        
        items.append(item)


In [6]:
# search all branches in Korea
response = starbucks.query(
    IndexName='GSI_01_Locations',
    KeyConditionExpression='Country = :country',
    ExpressionAttributeValues={
        ':country': 'KR'
    }
)

pprint(response)

{'Count': 0,
 'Items': [],
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '39',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sun, 04 Oct 2020 18:06:30 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '3413411624',
                                      'x-amzn-requestid': '5OR4D0H5M0CVI2AB17R1MIVEC3VV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': '5OR4D0H5M0CVI2AB17R1MIVEC3VV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0},
 'ScannedCount': 0}


In [4]:
starbucks.global_secondary_indexes

[{'IndexName': 'GSI_01_Locations',
  'KeySchema': [{'AttributeName': 'Country', 'KeyType': 'HASH'},
   {'AttributeName': 'StateCity', 'KeyType': 'RANGE'}],
  'Projection': {'ProjectionType': 'ALL'},
  'IndexStatus': 'ACTIVE',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 0,
   'WriteCapacityUnits': 0},
  'IndexSizeBytes': 0,
  'ItemCount': 0,
  'IndexArn': 'arn:aws:dynamodb:ap-northeast-2:886100642687:table/Starbucks/index/GSI_01_Locations'}]